In [ ]:
# | default_exp _testing.in_memory_broker

In [ ]:
# | export
import uuid
from collections import namedtuple
from dataclasses import dataclass
from contextlib import contextmanager
import inspect
import asyncio
import copy
import random
import hashlib
import string

from typing import *
import fastkafka._application.app
import fastkafka._components.aiokafka_consumer_loop
import fastkafka._components.aiokafka_producer_manager
from aiokafka import AIOKafkaConsumer, AIOKafkaProducer
from aiokafka.structs import ConsumerRecord, TopicPartition, RecordMetadata

import fastkafka._application.app
from fastkafka._components.meta import copy_func, patch, delegates, classcontextmanager, _get_default_kwargs_from_sig
from fastkafka._components.logger import get_logger

In [ ]:
from contextlib import asynccontextmanager
import unittest

import pytest

from fastkafka.testing import ApacheKafkaBroker

In [ ]:
# | export

logger = get_logger(__name__)

# Local Kafka broker
> In-memory mockup of Kafka broker protocol

## Kafka partition

In [ ]:
# | export


@dataclass
class KafkaRecord:
    topic: str = ""
    partition: int = 0
    key: Optional[bytes] = None
    value: bytes = b""
    offset: int = 0

In [ ]:
# | export


class KafkaPartition:
    def __init__(self, *, partition: int, topic: str):
        self.partition = partition
        self.topic = topic
        self.messages: List[KafkaRecord] = list()

    def write(self, value: bytes, key: Optional[bytes] = None) -> RecordMetadata: # type: ignore
        record = KafkaRecord(
            topic=self.topic,
            partition=self.partition,
            value=value,
            key=key,
            offset=len(self.messages),
        )
        record_meta = RecordMetadata(
            topic=self.topic,
            partition=self.partition,
            topic_partition=TopicPartition(topic=self.topic, partition=self.partition),
            offset=len(self.messages),
            timestamp=1680602752070,
            timestamp_type=0,
            log_start_offset=0,
        )
        self.messages.append(record)
        return record_meta

    def read(self, offset: int) -> Tuple[List[KafkaRecord], int]:
        return self.messages[offset:], len(self.messages)

    def latest_offset(self) -> int:
        return len(self.messages)

In [ ]:
partition_index = 0
topic = "test"
partition = KafkaPartition(partition=partition_index, topic=topic)

msgs = [b"some_msg" for _ in range(25)]

expected = [
    KafkaRecord(topic=topic, partition=partition_index, value=msg, offset=offset)
    for offset, msg in enumerate(msgs)
]

for msg in msgs:
    partition.write(msg)

for offset in [0, 10, 20]:
    actual = partition.read(offset=offset)

    assert actual == (expected[offset:], len(msgs))

In [ ]:
partition_index = 0
topic = "test"
key = b"some_key"
partition = KafkaPartition(partition=partition_index, topic=topic)

msgs = [b"some_msg" for _ in range(25)]
expected = [
    KafkaRecord(
        topic=topic, partition=partition_index, value=msg, key=key, offset=offset
    )
    for offset, msg in enumerate(msgs)
]

for msg in msgs:
    partition.write(msg, key=key)

for offset in [0, 10, 20]:
    actual = partition.read(offset=offset)

    assert actual == (expected[offset:], len(msgs)), print(f"{actual} != {expected}")

## Kafka topic

In [ ]:
# | export


class KafkaTopic:
    def __init__(self, topic: str, num_partitions: int = 1):
        self.topic = topic
        self.num_partitions = num_partitions
        self.partitions: List[KafkaPartition] = [
            KafkaPartition(topic=topic, partition=partition_index)
            for partition_index in range(num_partitions)
        ]

    def read(  # type: ignore
        self, partition: int, offset: int
    ) -> Tuple[TopicPartition, List[KafkaRecord], int]:
        topic_partition = TopicPartition(topic=self.topic, partition=partition)
        records, offset = self.partitions[partition].read(offset)
        return topic_partition, records, offset

    def write_with_partition(  # type: ignore
        self,
        value: bytes,
        partition: int,
    ) -> RecordMetadata:
        return self.partitions[partition].write(value)

    def write_with_key(self, value: bytes, key: bytes) -> RecordMetadata:  # type: ignore
        partition = int(hashlib.sha256(key).hexdigest(), 16) % self.num_partitions
        return self.partitions[partition].write(value, key=key)

    def write(  # type: ignore
        self,
        value: bytes,
        *,
        key: Optional[bytes] = None,
        partition: Optional[int] = None,
    ) -> RecordMetadata:
        if partition is not None:
            return self.write_with_partition(value, partition)

        if key is not None:
            return self.write_with_key(value, key)

        partition = random.randint(0, self.num_partitions - 1)  # nosec
        return self.write_with_partition(value, partition)

    def latest_offset(self, partition: int) -> int:
        return self.partitions[partition].latest_offset()

In [ ]:
msg = b"msg"

topic = KafkaTopic("test_topic", 1)

expected = RecordMetadata(
    topic="test_topic",
    partition=0,
    topic_partition=TopicPartition(topic="test_topic", partition=0),
    offset=0,
    timestamp=1680602752070,
    timestamp_type=0,
    log_start_offset=0,
)
actual = topic.write(msg)

assert expected == actual

expected = RecordMetadata(
    topic="test_topic",
    partition=0,
    topic_partition=TopicPartition(topic="test_topic", partition=0),
    offset=1,
    timestamp=1680602752070,
    timestamp_type=0,
    log_start_offset=0,
)
actual = topic.write(msg, key=b"123")

assert expected == actual, actual

In [ ]:
topic_name = "test_topic"
msgs = [b"msg" for _ in range(1000)]
partition_num = 10

topic = KafkaTopic(topic_name, partition_num)

# write to topic
for msg in msgs:
    topic.write(msg)

# For each partition in topic check:
for partition in range(partition_num):
    topic_partition_expected = TopicPartition(topic=topic_name, partition=partition)
    topic_partition_actual, data, _ = topic.read(partition=partition, offset=0)
    
    # Read returns correct TopicPartition key
    assert topic_partition_actual == topic_partition_expected
    
    # Data is written into partition
    assert len(data) > 0

In [ ]:
topic_name = "test_topic"
msgs = [b"msg" for _ in range(1000)]
partition_num = 2

topic = KafkaTopic(topic_name, partition_num)

# write to topic with defined partition
for msg in msgs:
    topic.write(msg, partition=0)

lengths = [
    len(
        topic.read(partition=i, offset=0)[1]
    )
    for i in range(partition_num)
]

assert [1000, 0] == lengths

In [ ]:
topic_name = "test_topic"
msgs = [b"msg" for _ in range(1000)]
partition_num = 3

topic = KafkaTopic(topic_name, partition_num)

# write to topic with defined key
for msg in msgs[:450]:
    topic.write(msg, key=b"some_key")

for msg in msgs[450:]:
    topic.write(msg, key=b"some_key443")

lengths = [
    len(
        topic.read(partition=i, offset=0)[1]
    )
    for i in range(partition_num)
]

assert [0, 450, 550] == sorted(lengths)

## Group metadata

In [ ]:
# | export


def split_list(list_to_split: List[Any], split_size: int) -> List[List[Any]]:
    return [
        list_to_split[start_index : start_index + split_size]
        for start_index in range(0, len(list_to_split), split_size)
    ]

In [ ]:
assert split_list([1, 2, 3, 4, 5], 1) == [[1], [2], [3], [4], [5]]
assert split_list([1, 2, 3, 4, 5], 2) == [[1, 2], [3, 4], [5]]
assert split_list([1, 2, 3, 4, 5], 3) == [[1, 2, 3], [4, 5]]
assert split_list([1, 2, 3, 4, 5], 5) == [[1, 2, 3, 4, 5]]

In [ ]:
# | export


class GroupMetadata:
    def __init__(self, num_partitions: int):
        self.num_partitions = num_partitions
        self.partitions_offsets: Dict[int, int] = {}
        self.consumer_ids: List[uuid.UUID] = list()
        self.partition_assignments: Dict[uuid.UUID, List[int]] = {}

    def subscribe(self, consumer_id: uuid.UUID) -> None:
        self.consumer_ids.append(consumer_id)
        self.rebalance()

    def unsubscribe(self, consumer_id: uuid.UUID) -> None:
        self.consumer_ids.remove(consumer_id)
        self.rebalance()

    def rebalance(self) -> None:
        if len(self.consumer_ids) == 0:
            self.partition_assignments = {}
        else:
            partitions_per_actor = self.num_partitions // len(self.consumer_ids)
            if self.num_partitions % len(self.consumer_ids) != 0:
                partitions_per_actor += 1
            self.assign_partitions(partitions_per_actor)

    def assign_partitions(self, partitions_per_actor: int) -> None:
        partitions = [i for i in range(self.num_partitions)]

        partitions_split = split_list(partitions, partitions_per_actor)
        self.partition_assignments = {
            self.consumer_ids[i]: partition_split
            for i, partition_split in enumerate(partitions_split)
        }

    def get_partitions(
        self, consumer_id: uuid.UUID
    ) -> Tuple[List[int], Dict[int, Optional[int]]]:
        partition_assignments = self.partition_assignments.get(consumer_id, [])
        partition_offsets_assignments = {
            partition: self.partitions_offsets.get(partition, None)
            for partition in partition_assignments
        }
        return partition_assignments, partition_offsets_assignments

    def set_offset(self, partition: int, offset: int) -> None:
        self.partitions_offsets[partition] = offset

In [ ]:
group_meta = GroupMetadata(num_partitions=3)

# subscribe first consumer
consumer_id_1 = uuid.uuid4()
group_meta.subscribe(consumer_id_1)
# check partitions
assert group_meta.get_partitions(consumer_id_1)[0] == [0, 1, 2]

# subscribe second consumer
consumer_id_2 = uuid.uuid4()
group_meta.subscribe(consumer_id_2)
# check partitions
assert group_meta.get_partitions(consumer_id_1)[0] == [0, 1]
assert group_meta.get_partitions(consumer_id_2)[0] == [2]

# subscribe third consumer
consumer_id_3 = uuid.uuid4()
group_meta.subscribe(consumer_id_3)
# check partitions
assert group_meta.get_partitions(consumer_id_1)[0] == [0]
assert group_meta.get_partitions(consumer_id_2)[0] == [1]
assert group_meta.get_partitions(consumer_id_3)[0] == [2]

# subscribe fourth consumer
# subscribe third consumer
consumer_id_4 = uuid.uuid4()
group_meta.subscribe(consumer_id_4)
# check partitions
assert group_meta.get_partitions(consumer_id_1)[0] == [0]
assert group_meta.get_partitions(consumer_id_2)[0] == [1]
assert group_meta.get_partitions(consumer_id_3)[0] == [2]
assert group_meta.get_partitions(consumer_id_4)[0] == [] # fourth consumer is starving

# Unsubscribe one consumer
group_meta.unsubscribe(consumer_id_3)
# check partitions
assert group_meta.get_partitions(consumer_id_1)[0] == [0]
assert group_meta.get_partitions(consumer_id_2)[0] == [1]
assert group_meta.get_partitions(consumer_id_4)[0] == [2], group_meta.get_partitions(consumer_id_4)

# Unsubscribe all but one consumer
group_meta.unsubscribe(consumer_id_1)
group_meta.unsubscribe(consumer_id_4)
assert group_meta.get_partitions(consumer_id_2)[0] == [0, 1, 2]


## Kafka broker

In [ ]:
# | export


@classcontextmanager()
class InMemoryBroker:
    def __init__(self, topics: Iterable[str], num_partitions: int = 1):
        self.num_partitions = num_partitions
        self.topics: Dict[str, KafkaTopic] = {
            topic: KafkaTopic(topic, num_partitions) for topic in topics
        }
        self.topic_groups: Dict[Tuple[str, str], GroupMetadata] = {}
        self.is_started: bool = False

    def connect(self) -> uuid.UUID:
        return uuid.uuid4()

    def dissconnect(self, consumer_id: uuid.UUID) -> None:
        pass

    def subscribe(self, topic: str, group: str, consumer_id: uuid.UUID) -> None:
        group_meta = self.topic_groups.get(
            (topic, group), GroupMetadata(self.num_partitions)
        )
        group_meta.subscribe(consumer_id)
        self.topic_groups[(topic, group)] = group_meta

    def unsubscribe(self, topic: str, group: str, consumer_id: uuid.UUID) -> None:
        self.topic_groups[(topic, group)].unsubscribe(consumer_id)

    def read(  # type: ignore
        self, *, topic: str, group: str, consumer_id: uuid.UUID, auto_offset_reset: str
    ) -> Dict[TopicPartition, List[KafkaRecord]]:
        group_meta = self.topic_groups[(topic, group)]
        partitions, offsets = group_meta.get_partitions(consumer_id)

        if len(partitions) == 0:
            return {}

        partitions_data = {}

        for partition in partitions:
            offset = offsets[partition]

            if offset is None:
                offset = (
                    self.topics[topic].latest_offset(partition)
                    if auto_offset_reset == "latest"
                    else 0
                )

            topic_partition, data, offset = self.topics[topic].read(partition, offset)

            partitions_data[topic_partition] = data
            group_meta.set_offset(partition, offset)

        return partitions_data

    def write(  # type: ignore
        self,
        *,
        topic: str,
        value: bytes,
        key: Optional[bytes] = None,
        partition: Optional[int] = None,
    ) -> RecordMetadata:
        if topic in self.topics:
            return self.topics[topic].write(value, key=key, partition=partition)
        else:
            # todo: log only once
            logger.warning(
                f"Topic {topic} is not available during auto-create initialization"
            )

    @contextmanager
    def lifecycle(self) -> Iterator["InMemoryBroker"]:
        raise NotImplementedError()

    async def _start(self) -> str:
        logger.info("InMemoryBroker._start() called")
        self.__enter__()  # type: ignore
        return "localbroker:0"

    async def _stop(self) -> None:
        logger.info("InMemoryBroker._stop() called")
        self.__exit__(None, None, None)  # type: ignore

In [ ]:
# Check subscribing and reading from empty partitions for same group

broker = InMemoryBroker(topics=["topic1"], num_partitions=3)

consumer_group = "my_group"

consumer_id_1 = broker.connect()
broker.subscribe("topic1", consumer_group, consumer_id_1)

assert broker.read(
    topic="topic1",
    group=consumer_group,
    consumer_id=consumer_id_1,
    auto_offset_reset="latest",
) == {
    TopicPartition(topic="topic1", partition=0): [],
    TopicPartition(topic="topic1", partition=1): [],
    TopicPartition(topic="topic1", partition=2): [],
}

consumer_id_2 = broker.connect()
broker.subscribe("topic1", consumer_group, consumer_id_2)

assert broker.read(
    topic="topic1",
    group=consumer_group,
    consumer_id=consumer_id_1,
    auto_offset_reset="latest",
) == {
    TopicPartition(topic="topic1", partition=0): [],
    TopicPartition(topic="topic1", partition=1): [],
}

assert broker.read(
    topic="topic1",
    group=consumer_group,
    consumer_id=consumer_id_2,
    auto_offset_reset="latest",
) == {
    TopicPartition(topic="topic1", partition=2): [],
}

broker.unsubscribe("topic1", consumer_group, consumer_id_1)
assert broker.read(
    topic="topic1",
    group=consumer_group,
    consumer_id=consumer_id_2,
    auto_offset_reset="latest",
) == {
    TopicPartition(topic="topic1", partition=0): [],
    TopicPartition(topic="topic1", partition=1): [],
    TopicPartition(topic="topic1", partition=2): [],
}

In [ ]:
# check writing to partitions

topic_name = "topic1"

broker = InMemoryBroker(topics=[topic_name], num_partitions=1)

consumer_group = "my_group"

consumer_id_1 = broker.connect()
broker.subscribe(topic_name, consumer_group, consumer_id_1)

record_meta = broker.write(topic=topic_name, value=b"msg")

assert record_meta == RecordMetadata(
    topic=topic_name,
    partition=0,
    topic_partition=TopicPartition(topic=topic_name, partition=0),
    offset=0,
    timestamp=1680602752070,
    timestamp_type=0,
    log_start_offset=0,
)

assert broker.read(
    topic=topic_name,
    consumer_id=consumer_id_1,
    group=consumer_group,
    auto_offset_reset="earliest",
) == {
    TopicPartition(topic="topic1", partition=0): [
        KafkaRecord(topic="topic1", partition=0, key=None, value=b"msg", offset=0)
    ]
}

broker.write(topic=topic_name, value=b"msg")

consumer_group_new = "another_group"

consumer_id_2 = broker.connect()
broker.subscribe(topic_name, consumer_group_new, consumer_id_2)

assert broker.read(
    topic=topic_name,
    consumer_id=consumer_id_2,
    group=consumer_group_new,
    auto_offset_reset="latest",
) == {TopicPartition(topic="topic1", partition=0): []}

assert broker.read(
    topic=topic_name,
    consumer_id=consumer_id_1,
    group=consumer_group,
    auto_offset_reset="latest",
) == {
    TopicPartition(topic="topic1", partition=0): [
        KafkaRecord(topic="topic1", partition=0, key=None, value=b"msg", offset=1)
    ]
}

In [ ]:
in_memory_broker = InMemoryBroker(["my_topic"])
group = "my_group"

consumer_id = in_memory_broker.connect()

with pytest.raises(KeyError) as e:
    in_memory_broker.read(topic="my_topic", group=group, consumer_id=consumer_id, auto_offset_reset="latest")

in_memory_broker.subscribe(topic="my_topic", group=group, consumer_id=consumer_id)

msg = in_memory_broker.read(topic="my_topic", group=group, consumer_id=consumer_id, auto_offset_reset="earliest")
assert msg == {TopicPartition(topic='my_topic', partition=0): []}, msg

in_memory_broker.write(topic="not_my_topic", value=b"not my message")

[WARNING] __main__: Topic not_my_topic is not available during auto-create initialization


In [ ]:
#| notest

with ApacheKafkaBroker(["my_topic"], apply_nest_asyncio=True) as bootstrap_servers:
    producer = AIOKafkaProducer(bootstrap_servers=bootstrap_servers)
    await producer.start()
    for _ in range(1000):
        record = await producer.send(topic="not_my_topic", value=b"not my message")
    await producer.stop()

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.sta

In [ ]:
#| notest

with ApacheKafkaBroker(["my_topic"], apply_nest_asyncio=True) as bootstrap_servers:
    consumer = AIOKafkaConsumer("my_topic", bootstrap_servers=bootstrap_servers)
    await consumer.start()
    print("getmany()...")
    msg = await consumer.getmany(timeout_ms=0)
    print("exiting...")
    await consumer.stop()

[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): entering...
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): (<_UnixSelectorEventLoop running=True closed=False debug=False>) is already running!
[WARNING] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): calling nest_asyncio.apply()
[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: <class 'fastkafka.testing.ApacheKafkaBroker'>.start(): returning 127.0.0.1:9092
[INFO] fastkafka._testing.apache_kafka_broker: ApacheKafkaBroker.start(): exited.
[INFO] aiokafka.consumer.subscription_state: Updating subscri

## Consumer patching

We need to patch AIOKafkaConsumer methods so that we can redirect the consumer to our local kafka broker.

Patched methods:

- [x] \_\_init\_\_
- [x] start
- [x] subscribe
- [x] stop
- [x] getmany

In [ ]:
# | export


# InMemoryConsumer
class InMemoryConsumer:
    def __init__(
        self,
        broker: InMemoryBroker,
    ) -> None:
        self.broker = broker
        self._id: Optional[uuid.UUID] = None
        self._auto_offset_reset: str = "latest"
        self._group_id: Optional[str] = None
        self._topics: List[str] = list()

    @delegates(AIOKafkaConsumer)
    def __call__(self, **kwargs: Any) -> "InMemoryConsumer":
        defaults = _get_default_kwargs_from_sig(InMemoryConsumer.__call__, **kwargs)
        consume_copy = InMemoryConsumer(self.broker)
        consume_copy._auto_offset_reset = defaults["auto_offset_reset"]
        consume_copy._group_id = (
            defaults["group_id"]
            if defaults["group_id"] is not None
            else "".join(random.choices(string.ascii_letters, k=10)) # nosec
        )
        return consume_copy

    @delegates(AIOKafkaConsumer.start)
    async def start(self, **kwargs: Any) -> None:
        pass

    @delegates(AIOKafkaConsumer.stop)
    async def stop(self, **kwargs: Any) -> None:
        pass

    @delegates(AIOKafkaConsumer.subscribe)
    def subscribe(self, topics: List[str], **kwargs: Any) -> None:
        raise NotImplementedError()

    @delegates(AIOKafkaConsumer.getmany)
    async def getmany(  # type: ignore
        self, **kwargs: Any
    ) -> Dict[TopicPartition, List[ConsumerRecord]]:
        raise NotImplementedError()

In [ ]:
broker = InMemoryBroker(["topic"])

ConsumerClass = InMemoryConsumer(broker)

for cls in [ConsumerClass, AIOKafkaConsumer]:
    consumer = cls()
    assert consumer._auto_offset_reset == "latest"

    consumer = cls(auto_offset_reset="earliest")
    assert consumer._auto_offset_reset == "earliest", consumer._auto_offset_reset

    consumer = cls(auto_offset_reset="whatever")
    assert consumer._auto_offset_reset == "whatever"
    
    await consumer.stop()
    

[ERROR] asyncio: Unclosed AIOKafkaConsumer
consumer: <aiokafka.consumer.consumer.AIOKafkaConsumer object>
[ERROR] asyncio: Unclosed AIOKafkaConsumer
consumer: <aiokafka.consumer.consumer.AIOKafkaConsumer object>


Patching start so that we don't try to start the real AIOKafkaConsumer instance

In [ ]:
# | export


@patch
@delegates(AIOKafkaConsumer.start)
async def start(self: InMemoryConsumer, **kwargs: Any) -> None:
    logger.info("AIOKafkaConsumer patched start() called()")
    if self._id is not None:
        raise RuntimeError(
            "Consumer start() already called! Run consumer stop() before running start() again"
        )
    self._id = self.broker.connect()

In [ ]:
broker = InMemoryBroker(["my_topic"])

ConsumerClass = InMemoryConsumer(broker)

for cls in [ConsumerClass]:

    consumer = cls()
    await consumer.start()
    await consumer.stop()

[INFO] __main__: AIOKafkaConsumer patched start() called()


Patching subscribe so that we can connect to our Local, in-memory, Kafka broker

In [ ]:
# | export


@patch
@delegates(AIOKafkaConsumer.subscribe)
def subscribe(self: InMemoryConsumer, topics: List[str], **kwargs: Any) -> None:
    logger.info("AIOKafkaConsumer patched subscribe() called")
    if self._id is None:
        raise RuntimeError("Consumer start() not called! Run consumer start() first")
    logger.info(f"AIOKafkaConsumer.subscribe(), subscribing to: {topics}")
    for topic in topics:
        self.broker.subscribe(consumer_id=self._id, topic=topic, group=self._group_id)  # type: ignore
        self._topics.append(topic)

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])

ConsumerClass = InMemoryConsumer(broker)
consumer = ConsumerClass()

await consumer.start()
consumer.subscribe(["my_topic"])

[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['my_topic']


Patching stop so that be dont break anything by calling the real AIOKafkaConsumer stop()

In [ ]:
# | export

@patch
@delegates(AIOKafkaConsumer.stop)
async def stop(self: InMemoryConsumer, **kwargs: Any) -> None:
    logger.info("AIOKafkaConsumer patched stop() called")
    if self._id is None:
        raise RuntimeError(
            "Consumer start() not called! Run consumer start() first"
        )
    for topic in self._topics:
        self.broker.unsubscribe(topic=topic, group=self._group_id, consumer_id=self._id)  # type: ignore

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])

ConsumerClass = InMemoryConsumer(broker)
consumer = ConsumerClass()

await consumer.start()
consumer.subscribe(["my_topic"])
await consumer.stop()

[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['my_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called


Patching getmany so that the messages are pulled from our Local, in-memory, Kafka broker

In [ ]:
# | export


@patch
@delegates(AIOKafkaConsumer.getmany)
async def getmany(  # type: ignore
    self: InMemoryConsumer, **kwargs: Any
) -> Dict[TopicPartition, List[ConsumerRecord]]:
    for topic in self._topics:
        return self.broker.read(
            topic=topic,
            consumer_id=self._id,  # type: ignore
            group=self._group_id,  # type: ignore
            auto_offset_reset=self._auto_offset_reset,
        )

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])

ConsumerClass = InMemoryConsumer(broker)
consumer = ConsumerClass(auto_offset_reset="latest")

await consumer.start()

consumer.subscribe(["my_topic"])
await consumer.getmany()

await consumer.stop()

[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['my_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called


## Producer patching

We need to patch AIOKafkaProducer methods so that we can redirect the producer to our local kafka broker

- [x] \_\_init\_\_
- [x] start
- [x] stop
- [x] send

In [ ]:
# | export


class InMemoryProducer:
    def __init__(self, broker: InMemoryBroker, **kwargs: Any) -> None:
        self.broker = broker
        self.id: Optional[uuid.UUID] = None

    @delegates(AIOKafkaProducer)
    def __call__(self, **kwargs: Any) -> "InMemoryProducer":
        return InMemoryProducer(self.broker)

    @delegates(AIOKafkaProducer.start)
    async def start(self, **kwargs: Any) -> None:
        raise NotImplementedError()

    @delegates(AIOKafkaProducer.stop)
    async def stop(self, **kwargs: Any) -> None:
        raise NotImplementedError()

    @delegates(AIOKafkaProducer.send)
    async def send(  # type: ignore
        self: AIOKafkaProducer,
        topic: str,
        msg: bytes,
        key: Optional[bytes] = None,
        **kwargs: Any,
    ):
        raise NotImplementedError()

Patching AIOKafkaProducer start so that we mock the startup procedure of AIOKafkaProducer

In [ ]:
# | export


@patch # type: ignore
@delegates(AIOKafkaProducer.start)
async def start(self: InMemoryProducer, **kwargs: Any) -> None:
    logger.info("AIOKafkaProducer patched start() called()")
    if self.id is not None:
        raise RuntimeError(
            "Producer start() already called! Run producer stop() before running start() again"
        )
    self.id = self.broker.connect()

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

await producer.start()

[INFO] __main__: AIOKafkaProducer patched start() called()


Patching AIOKafkaProducerStop so that we don't uniintentionally try to stop a real instance of AIOKafkaProducer

In [ ]:
# | export


@patch # type: ignore
@delegates(AIOKafkaProducer.stop)
async def stop(self: InMemoryProducer, **kwargs: Any) -> None:
    logger.info("AIOKafkaProducer patched stop() called")
    if self.id is None:
        raise RuntimeError(
            "Producer start() not called! Run producer start() first"
        )

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

await producer.start()
await producer.stop()

[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaProducer patched stop() called


Patching AIOKafkaProducer send so that we redirect sent messages to Local, in-memory, Kafka broker

In [ ]:
# | export


@patch
@delegates(AIOKafkaProducer.send)
async def send( # type: ignore
    self: InMemoryProducer,
    topic: str,
    msg: bytes,
    key: Optional[bytes] = None,
    partition: Optional[int] = None,
    **kwargs: Any,
): #asyncio.Task[RecordMetadata]
    if self.id is None:
        raise RuntimeError(
            "Producer start() not called! Run producer start() first"
        )
    record = self.broker.write(topic=topic, value=msg, key=key, partition=partition)

    async def _f(record: ConsumerRecord = record) -> RecordMetadata: # type: ignore
        return record

    return asyncio.create_task(_f())

In [ ]:
broker = InMemoryBroker(topics=["my_topic"])

ProducerClass = InMemoryProducer(broker)
producer = ProducerClass()

await producer.start()
msg_fut = await producer.send("my_topic", b"some_msg")
await msg_fut

[INFO] __main__: AIOKafkaProducer patched start() called()


RecordMetadata(topic='my_topic', partition=0, topic_partition=TopicPartition(topic='my_topic', partition=0), offset=0, timestamp=1680602752070, timestamp_type=0, log_start_offset=0)

## Add patching to InMemoryBroker

In [ ]:
# | export


@patch
@contextmanager
def lifecycle(self: InMemoryBroker) -> Iterator[InMemoryBroker]:
    logger.info(
        "InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!"
    )
    try:
        logger.info("InMemoryBroker starting")
        
        old_consumer_app = fastkafka._application.app.AIOKafkaConsumer
        old_producer_app = fastkafka._application.app.AIOKafkaProducer
        old_consumer_loop = fastkafka._components.aiokafka_consumer_loop.AIOKafkaConsumer
        old_producer_manager = fastkafka._components.aiokafka_producer_manager.AIOKafkaProducer
        
        fastkafka._application.app.AIOKafkaConsumer = InMemoryConsumer(self)
        fastkafka._application.app.AIOKafkaProducer = InMemoryProducer(self)
        fastkafka._components.aiokafka_consumer_loop.AIOKafkaConsumer = InMemoryConsumer(self)
        fastkafka._components.aiokafka_producer_manager.AIOKafkaProducer = InMemoryProducer(self)
        
        self.is_started = True
        yield self
    finally:
        logger.info("InMemoryBroker stopping")
        
        fastkafka._application.app.AIOKafkaConsumer = old_consumer_app
        fastkafka._application.app.AIOKafkaProducer = old_producer_app
        fastkafka._components.aiokafka_consumer_loop.AIOKafkaConsumer = old_consumer_loop
        fastkafka._components.aiokafka_producer_manager.AIOKafkaProducer = old_producer_manager
        
        self.is_started = False

In [ ]:
assert fastkafka._application.app.AIOKafkaConsumer == AIOKafkaConsumer
assert fastkafka._application.app.AIOKafkaProducer == AIOKafkaProducer

with InMemoryBroker(["topic"]) as broker:
    assert isinstance(fastkafka._application.app.AIOKafkaConsumer, InMemoryConsumer)
    assert isinstance(fastkafka._application.app.AIOKafkaProducer, InMemoryProducer)
    assert fastkafka._application.app.AIOKafkaConsumer().broker == broker
    assert fastkafka._application.app.AIOKafkaProducer().broker == broker
    
assert fastkafka._application.app.AIOKafkaConsumer == AIOKafkaConsumer
assert fastkafka._application.app.AIOKafkaProducer == AIOKafkaProducer

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: InMemoryBroker stopping


## Broker, consumer and producer integration tests

In [ ]:
@asynccontextmanager
async def create_consumer_and_producer(
    auto_offset_reset: str = "latest",
) -> AsyncIterator[Tuple[AIOKafkaConsumer, AIOKafkaProducer]]:
    consumer = fastkafka._application.app.AIOKafkaConsumer(auto_offset_reset=auto_offset_reset)
    producer = fastkafka._application.app.AIOKafkaProducer()

    await consumer.start()
    await producer.start()

    yield (consumer, producer)

    await consumer.stop()
    await producer.stop()

In [ ]:
def checkEqual(L1, L2):
    return len(L1) == len(L2) and sorted(L1) == sorted(L2)

In [ ]:
assert checkEqual([1, 2], [3]) == False
assert checkEqual([1, 2, 3], [3, 2, 1]) == True

Sanity check, let's see if the messages are sent to broker and received by the consumer

In [ ]:
topic = "test_topic"
sent_msgs = [f"msg{i}".encode("UTF-8") for i in range(320)]

with InMemoryBroker([topic]) as broker:
    async with create_consumer_and_producer(auto_offset_reset="earliest") as (
        consumer,
        producer,
    ):
        [await producer.send(topic, msg) for msg in sent_msgs]
        consumer.subscribe([topic])
        received = await consumer.getmany()
        received_msgs = [msg.value for _, msgs in received.items() for msg in msgs]
    assert checkEqual(
        received_msgs, sent_msgs
    ), f"{sent_msgs=}\n{received_msgs=}\n{data=}"

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
[INFO] __main__: InMemoryBroker stopping


Check if only subscribed topic messages are received by the consumer

In [ ]:
topic1 = "test_topic1"
topic2 = "test_topic2"
sent_msgs_1 = [(f"msg{i}" + topic1).encode("UTF-8") for i in range(32)]
sent_msgs_2 = [(f"msg{i}" + topic2).encode("UTF-8") for i in range(32)]

with InMemoryBroker([topic1, topic2]) as broker:
    async with create_consumer_and_producer(auto_offset_reset="earliest") as (
        consumer,
        producer,
    ):
        [await producer.send(topic1, msg) for msg in sent_msgs_1]
        [await producer.send(topic2, msg) for msg in sent_msgs_2]

        consumer.subscribe([topic1])
        received = await consumer.getmany()
        received_msgs = [msg.value for _, msgs in received.items() for msg in msgs]

    assert checkEqual(sent_msgs_1, received_msgs)

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic1']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
[INFO] __main__: InMemoryBroker stopping


Check if msgs are received only after subscribing when auto_offset_reset is set to "latest"

In [ ]:
topic = "test_topic"
sent_msgs_before = [f"msg{i}".encode("UTF-8") for i in range(32)]
sent_msgs_after = [f"msg{i}".encode("UTF-8") for i in range(32, 64)]

with InMemoryBroker([topic]) as broker:
    async with create_consumer_and_producer() as (consumer, producer):
        [await producer.send(topic, msg) for msg in sent_msgs_before]

        consumer.subscribe([topic])
        received = await consumer.getmany()
        [await producer.send(topic, msg) for msg in sent_msgs_after]
        received = await consumer.getmany()
        received_msgs = [msg.value for _, msgs in received.items() for msg in msgs]

    assert checkEqual(sent_msgs_after, received_msgs)

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
[INFO] __main__: InMemoryBroker stopping


Check two consumers different groups

In [ ]:
topic = "test_topic"
sent_msgs = [f"msg{i}".encode("UTF-8") for i in range(32)]

with InMemoryBroker([topic]) as broker:
    consumer1 = fastkafka._application.app.AIOKafkaConsumer(auto_offset_reset="earliest")
    consumer2 = fastkafka._application.app.AIOKafkaConsumer(auto_offset_reset="earliest")
    producer = fastkafka._application.app.AIOKafkaProducer()

    await consumer1.start()
    await consumer2.start()
    await producer.start()

    [await producer.send(topic, msg) for msg in sent_msgs]

    consumer1.subscribe([topic])
    received1 = await consumer1.getmany()
    
    consumer2.subscribe([topic])
    received2 = await consumer2.getmany()

    received_msgs1 = [msg.value for _, msgs in received1.items() for msg in msgs]
    received_msgs2 = [msg.value for _, msgs in received2.items() for msg in msgs]

    await consumer1.stop()
    await consumer2.stop()
    await producer.stop()
    
    assert checkEqual(sent_msgs, received_msgs1), received_msgs1
    assert checkEqual(sent_msgs, received_msgs2)

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[b'msg0', b'msg1', b'msg2', b'msg3', b'msg4', b'msg5', b'msg6', b'msg7', b'msg8', b'msg9', b'msg10', b'msg11', b'msg12', b'msg13', b'msg14', b'msg15', b'msg16', b'msg17', b'msg18', b'msg19', b'msg20', b'msg21', b'msg22', b'msg23', b'msg24', b'msg25', b'msg26', b'msg27', b'msg28', b'msg29', b'msg30', b'msg31']
[b'msg0', b'msg1', b'msg2', b'msg3', b'msg4', b'msg5', b'msg6', b'msg7', b'msg8', b'msg9', b'

Check two consumers same group

In [ ]:
topic = "test_topic"
sent_msgs = [f"msg{i}".encode("UTF-8") for i in range(32)]

with InMemoryBroker([topic], 5) as broker:
    consumer1 = fastkafka._application.app.AIOKafkaConsumer(group_id="my_group", auto_offset_reset="earliest")
    consumer2 = fastkafka._application.app.AIOKafkaConsumer(group_id="my_group", auto_offset_reset="earliest")
    producer = fastkafka._application.app.AIOKafkaProducer()

    await consumer1.start()
    await consumer2.start()
    await producer.start()

    [await producer.send(topic, msg) for msg in sent_msgs]

    consumer1.subscribe([topic])
    consumer2.subscribe([topic])
    
    received1 = await consumer1.getmany()
    received2 = await consumer2.getmany()

    received_msgs1 = [msg.value for _, msgs in received1.items() for msg in msgs]
    received_msgs2 = [msg.value for _, msgs in received2.items() for msg in msgs]

    await consumer1.stop()
    await consumer2.stop()
    await producer.stop()
    
    assert checkEqual(sent_msgs, received_msgs1 + received_msgs2)

[INFO] __main__: InMemoryBroker._patch_consumers_and_producers(): Patching consumers and producers!
[INFO] __main__: InMemoryBroker starting
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched start() called()
[INFO] __main__: AIOKafkaProducer patched start() called()
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched subscribe() called
[INFO] __main__: AIOKafkaConsumer.subscribe(), subscribing to: ['test_topic']
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaConsumer patched stop() called
[INFO] __main__: AIOKafkaProducer patched stop() called
[b'msg0', b'msg1', b'msg12', b'msg14', b'msg17', b'msg20', b'msg22', b'msg23', b'msg25', b'msg26', b'msg27', b'msg29', b'msg3', b'msg30', b'msg31', b'msg4', b'msg6', b'msg8', b'msg9']
[b'msg10', b'msg11', b'msg13', b'msg15', b'msg16', b